## Assignment3 PartB
Use for various tasks with hugginface transformers api
for following tasks

Text classification

Named entity recognition

Question answering

Text summarization

Translation

Zero-shot classification

Computer vision

Audio

Table QA

## Setup

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers[sentencepiece]
!pip install wikipedia-api
!pip install optimum
!pip install sacremoses
!pip install transformers
!pip install torch
!pip install timm

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_i

In [5]:
from transformers import DetrFeatureExtractor, DetrForObjectDetection, Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from transformers import pipeline, ResNetForImageClassification
from optimum.exporters.tasks import TasksManager
import matplotlib.pyplot as plt
from torchvision import models, transforms
from scipy.io import wavfile
from IPython.display import Audio
import soundfile as sf
from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np
import textwrap
import requests
import librosa
import torch
import gc

## Text classification

In [6]:
text = " I absolutely loved the new movie I watched last night! The acting was superb, the storyline was gripping, and the cinematography was breathtaking. I was on the edge of my seat the entire time, and the ending left me feeling deeply satisfied. This is definitely one of the best films I've seen in a long time. I can't recommend it enough "

In [7]:
wrapper = textwrap.TextWrapper(width=80, break_long_words=False, break_on_hyphens=False)
print(wrapper.fill(text))

 I absolutely loved the new movie I watched last night! The acting was superb,
the storyline was gripping, and the cinematography was breathtaking. I was on
the edge of my seat the entire time, and the ending left me feeling deeply
satisfied. This is definitely one of the best films I've seen in a long time. I
can't recommend it enough


In [8]:
sentiment_pipeline = pipeline('text-classification')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [9]:
sentiment_pipeline(text)


[{'label': 'POSITIVE', 'score': 0.9998763799667358}]

##Named entity recognition

In [37]:
named_entity_recognition_pipeline = pipeline("ner", model="dslim/bert-base-NER")


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [58]:
ner_text = """
During the World Health Organization's briefing in Geneva on March 11, 2020, Dr. Tedros Adhanom Ghebreyesus declared COVID-19 a global pandemic. This announcement came after the virus had spread to over 114 countries, causing significant loss of life and unprecedented disruptions to societies worldwide. In response, the European Union pledged €750 billion to support member states affected by the pandemic. The funds are aimed at healthcare systems, economic recovery, and assisting in vaccine distribution efforts. Notably, Pfizer and BioNTech were among the first to receive approval for their vaccine, which began distribution in December 2020. """

In [59]:
entities = named_entity_recognition_pipeline(ner_text, aggregation_strategy="simple")


In [60]:
def print_entity_details(entities):
    for entity in entities:
        print(f"EntityGroup: {entity['entity_group']} \tScore: {entity['score']:.2f} \tWord: {entity['word']}")


In [61]:
entities = named_entity_recognition_pipeline(ner_text, aggregation_strategy="simple")

In [62]:
print_entity_details(entities)

EntityGroup: ORG 	Score: 1.00 	Word: World Health Organization
EntityGroup: LOC 	Score: 1.00 	Word: Geneva
EntityGroup: PER 	Score: 1.00 	Word: Ted
EntityGroup: PER 	Score: 0.94 	Word: ##ros Adhanom Ghebreyesus
EntityGroup: MISC 	Score: 0.97 	Word: CO
EntityGroup: MISC 	Score: 0.54 	Word: ##D
EntityGroup: MISC 	Score: 0.57 	Word: 19
EntityGroup: ORG 	Score: 1.00 	Word: European Union
EntityGroup: ORG 	Score: 0.92 	Word: Pfizer
EntityGroup: ORG 	Score: 1.00 	Word: B
EntityGroup: ORG 	Score: 1.00 	Word: ##ioNTech


## Question answering

In [63]:
question_ans_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [73]:
import wikipediaapi


In [77]:
import wikipediaapi

# Set up custom identification for accessing Wikipedia's API
AGENT_IDENTIFIER = "dhshah1112/1.0 (dhshah1112@gmail.com)"

def fetch_wiki_article(title):
    # Establish connection to Wikipedia's API using a specific agent
    wikipedia_connection = wikipediaapi.Wikipedia(language='en', user_agent=AGENT_IDENTIFIER)
    article = wikipedia_connection.page(title)
    return article


In [78]:
title = "Sachin Tendulkar"

page = fetch_wiki_article(title)
text = page.text

print(text[:500])  # Print the first 500 characters of the article


Sachin Ramesh Tendulkar, ( ; pronounced [sətɕin teːɳɖulkəɾ]; born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest batsmen in the history of cricket. Hailed as the world's most prolific batsman of all time, he is the all-time highest run-scorer in both ODI and Test cricket with more than 18,000 runs and 15,000 runs, respectively. He also holds the record for receiving the most player of the match award


In [86]:
text_cleaned = text.replace("  ", " ")

In [79]:
def get_answer(output):
  return output['answer']

In [87]:
print(text_cleaned)

Sachin Ramesh Tendulkar, ( ; pronounced [sətɕin teːɳɖulkəɾ]; born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest batsmen in the history of cricket. Hailed as the world's most prolific batsman of all time, he is the all-time highest run-scorer in both ODI and Test cricket with more than 18,000 runs and 15,000 runs, respectively. He also holds the record for receiving the most player of the match awards in international cricket. Tendulkar was a Member of Parliament, Rajya Sabha by nomination from 2012 to 2018.Tendulkar took up cricket at the age of eleven, made his Test match debut on 15 November 1989 against Pakistan in Karachi at the age of sixteen, and went on to represent Mumbai domestically and India internationally for over 24 years. In 2002, halfway through his career, Wisden ranked him the second-greatest Test batsman of all time, behind Don Bradman, and the second-greatest ODI batsma

In [88]:
def qa(question):
  outputs = question_ans_pipeline(question=question, context=text_cleaned)
  qa_string = f"Q: {question}\nA: {get_answer(outputs)}"
  return qa_string


In [89]:
question = "When and why did Sachin Tendulkar let go of Indian captaincy?"

In [90]:
print(qa(question))

Q: When and why did Sachin Tendulkar let go of Indian captaincy?
A: allegations of conflict of interest


##Text summarization

In [91]:
summ_pipeline = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [96]:
text_summarize = text[:1000]

In [97]:
outputs = summ_pipeline(text_summarize, max_length=100, clean_up_tokenization_spaces=True)
print(wrapper.fill(outputs[0]['summary_text']))

 Sachin Ramesh Tendulkar is an Indian cricketer who captained the Indian
national team. He is widely regarded as one of the greatest batsmen in the
history of cricket. He holds the record for receiving the most player of the
match awards in international cricket. In 2002, Wisden ranked him the
second-greatest Test batsman of all time.


##Translation

In [98]:
trans_model = "Helsinki-NLP/opus-mt-en-es"

In [99]:
translation_pipeline = pipeline("translation", model=trans_model)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

In [ ]:
text_summarize

In [100]:
def translate(translation_text):
  translation_output = translator(text_sumarize, clean_up_tokenization_spaces=True, min_length=10)
  translated_text = translation_output[0]['translation_text']
  translation_string = f"Orig Text: \t{translation_text}\nTranslation: \t{translated_text}"
  return translation_string

In [102]:
print(translate(text_summarize))


NameError: name 'translator' is not defined

In [103]:
from transformers import pipeline

# Load a pre-trained model for translation, e.g., from English to French
translator = pipeline('translation_en_to_fr')

# Now you can use the translator in your function
def translate(translation_text):
    translation_output = translator(translation_text, clean_up_tokenization_spaces=True, min_length=10)
    translated_text = translation_output[0]['translation_text']
    translation_string = f"Orig Text: {translation_text}\nTranslation: {translated_text}"
    return translation_string


# Call the function with the text
print(translate(text_summarize))


No model was supplied, defaulted to google-t5/t5-base and revision 686f1db (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:171: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Orig Text: Sachin Ramesh Tendulkar, ( ; pronounced [sətɕin teːɳɖulkəɾ]; born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest batsmen in the history of cricket. Hailed as the world's most prolific batsman of all time, he is the all-time highest run-scorer in both ODI and Test cricket with more than 18,000 runs and 15,000 runs, respectively. He also holds the record for receiving the most player of the match awards in international cricket. Tendulkar was a Member of Parliament, Rajya Sabha by nomination from 2012 to 2018.Tendulkar took up cricket at the age of eleven, made his Test match debut on 15 November 1989 against Pakistan in Karachi at the age of sixteen, and went on to represent Mumbai domestically and India internationally for over 24 years. In 2002, halfway through his career, Wisden ranked him the second-greatest Test batsman of all time, behind Don Bradman, and the second-greatest

##Zero-shot classification

In [104]:
zero_shot_classifier = pipeline("zero-shot-classification",
                                model="vicgalle/xlm-roberta-large-xnli-anli")

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [105]:
text = "The launch of the James Webb Space Telescope has been a monumental success. As it orbits a million miles from Earth, scientists are eager to see the new data on distant galaxies."

In [106]:
classes = ['Treffen', 'Arbeit', 'Digital', 'Reisen']

In [107]:
zero_shot_classifier(text, classes, multi_label=True)


{'sequence': 'The launch of the James Webb Space Telescope has been a monumental success. As it orbits a million miles from Earth, scientists are eager to see the new data on distant galaxies.',
 'labels': ['Digital', 'Arbeit', 'Treffen', 'Reisen'],
 'scores': [0.7902402877807617,
  0.0022949178237468004,
  0.0015195427695289254,
  0.0006295379134826362]}

##Computer vision

In [ ]:
https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.shutterstock.com%2Fsearch%2Fcar-bike-pedestrian&psig=AOvVaw0Kso8Sz5lhOGRIBtaXytAl&ust=1711334661386000&source=images&cd=vfe&opi=89978449&ved=0CBIQjRxqFwoTCNDB9cnwi4UDFQAAAAAdAAAAABAE

In [110]:
import requests
from PIL import Image
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from io import BytesIO

# Load a pre-trained model with updated API usage
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model.eval()

# Define the transform
transform = transforms.Compose([
    transforms.ToTensor(),
])

# URL of an example image from a free stock image provider
image_url = "https://plus.unsplash.com/premium_photo-1680667682571-b030c3076b8c?q=80&w=2835&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"

# Fetch image from URL
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

# Transform the image to be compatible with the model
image = transform(image).unsqueeze(0)  # Add batch dimension

# Perform object detection
predictions = model(image)

# Since we cannot show the image with the bounding boxes in this environment,
# let's print the predictions to see what the model found
print(predictions)


[{'boxes': tensor([[1756.7657, 3433.7021, 1853.8993, 3524.1147],
        [   9.4753, 2515.5908,  259.6817, 2600.0745],
        [1547.0911, 3535.4302, 1671.1046, 3634.5793],
        [1870.5902, 3003.5676, 1936.6056, 3061.8008],
        [1238.1605, 3384.5549, 1351.5891, 3469.4153],
        [1532.7955, 2808.8196, 1585.9900, 2845.0676],
        [1648.5149, 2868.2654, 1714.9736, 2925.2244],
        [1745.9990, 2667.7688, 1798.8597, 2705.2654],
        [1714.2803, 2718.8069, 1764.8118, 2770.3418],
        [1392.1550, 3395.3154, 1492.4910, 3482.5066],
        [1492.0502, 3216.8191, 1581.8110, 3288.4280],
        [1754.9987, 2582.3770, 1809.8097, 2625.5342],
        [1946.3147, 3351.9707, 2021.4910, 3411.4253],
        [1661.3411, 3311.4224, 1748.7350, 3388.0896],
        [1846.8732, 2699.7212, 1899.2738, 2739.1765],
        [1796.0057, 2491.1272, 1847.9043, 2526.1636],
        [1515.3859, 2682.5837, 1551.1171, 2721.5076],
        [1799.0460, 2503.7434, 1845.9752, 2534.1638],
        [1756.646

In [114]:
import requests
from PIL import Image
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from io import BytesIO

# COCO class names for object detection (subset for brevity)
COCO_CLASSES = {
    1: 'grass', 2: 'Lion', 3: 'car', 4: 'motorcycle', 5: 'airplane',
    6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light',
    11: 'sun', 12 : 'Lion Cub'

}

# Load the pre-trained model
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model.eval()

# Define the transform
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Direct URL to the image file
image_url = "https://images.unsplash.com/photo-1583587067350-2c49115673c9?q=80&w=2940&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"

# Fetch the image from the URL
response = requests.get(image_url)
if response.status_code == 200:
    image = Image.open(BytesIO(response.content))

    # Transform the image
    image_tensor = transform(image).unsqueeze(0)

    # Perform object detection
    with torch.no_grad():
        predictions = model(image_tensor)

    # Display predictions with class names and confidence scores
    for label, box, score in zip(predictions[0]['labels'], predictions[0]['boxes'], predictions[0]['scores']):
        if score > 0.5:  # Filter out low-confidence detections
            class_name = COCO_CLASSES.get(label.item(), "Unknown")
            print(f"Class: {class_name}, Box: {box}, Score: {score.item():.4f}")
else:
    print("Failed to retrieve the image.")


Class: Unknown, Box: tensor([ 771.5916,  275.1300, 2879.0369, 1932.7056]), Score: 0.9031
Class: grass, Box: tensor([1855.5095, 1727.7507, 2482.8555, 1949.7548]), Score: 0.6068


##Audio

In [1]:
from scipy.io import wavfile
import numpy as np
import librosa
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
import torch
from IPython.display import Audio

In [20]:
path = '/content/drive/MyDrive/CMPE258/Assignment4_Dataset/0_01_0.wav'


In [21]:
Audio(path)

...

In [23]:
import numpy as np
from scipy.io import wavfile
import librosa
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
import torch
from IPython.display import Audio


In [24]:
# Define a function to load and preprocess the audio file
def preprocess_audio(file_path):
    # Use librosa to handle audio files other than wav
    audio, rate = librosa.load(file_path, sr=16000)
    duration = len(audio) / rate
    return audio, rate, duration

In [25]:
# Function to predict text from audio using a pretrained model
def predict_from_audio(audio, rate):
    # Initialize tokenizer and model
    tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
    model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

    # Prepare the audio for the model
    input_values = tokenizer(audio, return_tensors="pt", sampling_rate=rate).input_values

    # Retrieve logits and transcribe
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]

    return transcription


In [27]:
# Preprocess the audio
audio, rate, duration = preprocess_audio(path)
print(f"Sample Rate: {rate}")
print(f"Length of Recording: {duration} seconds")

Sample Rate: 16000
Length of Recording: 0.7474375 seconds


In [33]:

# Predict the transcription
transcription = predict_from_audio(audio, rate)
print("Text Extracted from audio :", transcription)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:733: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mod

Text Extracted from audio : CERO


In [34]:
# Play the audio file
Audio(path)

##Table QA

In [37]:
import pandas as pd
import seaborn as sns

# Load the Iris dataset
df_iris = sns.load_dataset('iris')

# Define a function to retrieve data based on a feature name
def get_iris_info(feature, dataframe):
    """
    Retrieves information from the Iris dataset based on a feature name.
    Parameters:
    - feature (str): Name of the feature to retrieve data for.
    - dataframe (pd.DataFrame): The DataFrame containing the Iris dataset.
    Returns:
    - pd.Series: A series from the DataFrame corresponding to the requested feature.
    """
    try:
        # Ensure the feature exists in the dataset
        if feature in dataframe.columns:
            return dataframe[feature]
        else:
            return "Feature not found in the dataset."
    except Exception as e:
        return str(e)

# Example usage of the function
example_feature = 'petal_length'
iris_info = get_iris_info(example_feature, df_iris)

# Display the first few entries for the feature
print(iris_info.head())


0    1.4
1    1.4
2    1.3
3    1.5
4    1.4
Name: petal_length, dtype: float64


In [38]:
import pandas as pd
import seaborn as sns

# Load the Iris dataset
df_iris = sns.load_dataset('iris')

# Define a simple QA function to answer questions about the average measurements
def answer_iris_query(query):
    """
    Answers simple questions about the average feature measurements of the Iris dataset.
    Parameters:
    - query (str): A natural language-like question about the Iris dataset features.
    Returns:
    - str: The answer to the query.
    """
    # Identify keywords in the query to determine the response
    if 'average' in query.lower():
        if 'sepal length' in query.lower():
            return f"The average sepal length is {df_iris['sepal_length'].mean():.2f} cm."
        elif 'sepal width' in query.lower():
            return f"The average sepal width is {df_iris['sepal_width'].mean():.2f} cm."
        elif 'petal length' in query.lower():
            return f"The average petal length is {df_iris['petal_length'].mean():.2f} cm."
        elif 'petal width' in query.lower():
            return f"The average petal width is {df_iris['petal_width'].mean():.2f} cm."
        else:
            return "Please specify which feature's average you're interested in: sepal length, sepal width, petal length, or petal width."
    else:
        return "Currently, I can only provide averages for sepal length, sepal width, petal length, and petal width."

# Example queries
queries = [
    "What is the average sepal length?",
    "Tell me the average petal width.",
    "What is the average petal length?",
    "How about the average sepal width?",
]

# Answer each query
for query in queries:
    print(f"Query: {query}")
    print(answer_iris_query(query), "\n")


Query: What is the average sepal length?
The average sepal length is 5.84 cm. 

Query: Tell me the average petal width.
The average petal width is 1.20 cm. 

Query: What is the average petal length?
The average petal length is 3.76 cm. 

Query: How about the average sepal width?
The average sepal width is 3.06 cm. 

